In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import urllib.request

In diesem Aufgabenblatt wollen wir ein recht einfaches Klassifikationsproblem einmal mit logistischer Regression (mit *Scikit-learn*) und dann mit einem Multi-Layer Perceptron (MLP) mit der Keras Sequential API untersuchen.
Der Datensatz enthält knapp 15.000 Einträge aus einer Mitarbeiter-Datenbank.
Die darin enthaltenen Informationen geben z.B. Auskunft darüber, wie lange eine Person schon in der Firma ist, wie hoch die Gehaltsstufe ist oder zufrieden die Person im Betrieb ist.
Als Label wollen wir die Information verwenden, ob die Person die Firma verlassen hat.

In [ ]:
url = "https://github.com/fh-swf-hgi/ml/raw/main/p6/MA.csv"
dfile = "./MA.csv"

if not os.path.isfile(dfile):
    urllib.request.urlretrieve(url, dfile)

data=pd.read_csv('MA.csv')
data.info()

Wir sehen, dass die meisten Merkmale numerisch sind, nur *Bereich* und *Gehalt* sind kategorisch.
Die Spalte Gehalt hat allerdings eine Sortierung, d.h. wir habe es hier eigentlich mit einem *ordinalen* Merkmal zu tun.
Daher übersetzen wir die Klassen (*high*, *low*, und *medium*) in ganze Zahlen.

In [ ]:
df = data.copy()
df['Gehalt'] = pd.Categorical(df['Gehalt'], categories=['high', 'medium', 'low'])
df['Gehalt'] = df.Gehalt.cat.codes

**Aufgabe: Transformieren Sie das Merkmal "Bereich" per One-Hot-Kodierung in numerische Merkmale. Selektieren Sie als Labels `y` die Spalte `Firma_verlassen` aus dem Datensatz. Der Datensatz `X` soll alle Spalten bis auf die Labels enthalten.**

*Hinweis:* Für die One-hot-Kodierung können Sie die Funktion `pandas.get_dummies` verwenden.

In [ ]:
y = None
X = None
# YOUR CODE HERE
raise NotImplementedError()
X.shape, y.shape

In [ ]:
assert X.shape == (14999, 18)
assert y.shape == (14999,)

**Aufgabe: Standardisieren Sie die Merkmale in `X`**

*Hinweis:* Sie können dazu die Klasse `StandardScale` aus `sklearn import preprocessing` verwenden.

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaled = None

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert all(np.isclose(X_scaled.mean(axis=0), 0, atol=1e-3))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

FEATURES = X_train.shape[1]
X_train.shape, X_test.shape

**Aufgabe: Trainieren Sie ein Logistic Regression Modell mit den Trainingsdaten. Berechnen Sie die Accuracy `ca` für die Testdaten.**

In [ ]:
from sklearn.linear_model import LogisticRegression
ca = None
# YOUR CODE HERE
raise NotImplementedError()
print("Accuracy = %.2f%%" % (ca*100))

In [ ]:
assert np.isclose(ca, 0.78888888888, atol=1e-4)

### MLP mit Keras

Wir wollen nun für die gleichen Daten ein Multi-Layer Perzeptron mit Keras aufstellen.
Dazu lesen wir zunächst die NumPy Arrays in Tensorflow `Datasets` ein.

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

Da wir das Mini-Batch Gradientenverfahren verwenden wollen, teilen wir unseren Datensatz in kleiner *Batches* auf.
Zuvor "mischen" wir die Datenpunkte durch, damit die Punkte in den einzelnen Batches möglichst unsortiert sind und damit ein einzelner Batch die Eigenschaften des kompletten Datensatzes möglichst gut repräsentiert.
Die Idee dahinter ist, das der Gradient, den wir für den Mini-Batch berechnen, möglichst ähnlich zu dem Gradienten der kompletten Daten ist.

In [ ]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

**Aufgabe: Stellen Sie nun ein sequentielles Keras Modell auf. Es soll mindestens 3 Schichten haben. Als Aktivierungsfunktion verwenden Sie `sigmoid`.**

In [ ]:
#Modell definieren
model = keras.Sequential()
model.add(keras.Input(shape=(FEATURES,)))

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert len(model.layers)>2, "Das Modell soll mindestens 3 Schichten haben"

Nun erzeugen wir das Modell mit der Funktion `compile`. Dabei geben wir das Optimierungsverfahren, die (Art der) Kostenfunktion sowie die zu berechnenden Metriken an.

**Aufgabe: Wählen Sie eine geeignet Kostenfunktion aus. Als Metrik soll die Accuracy berechnet werden.**

In [ ]:
#Modell erzeugen
optimizer = 'sgd'
loss = None
metrics = None

# YOUR CODE HERE
raise NotImplementedError()

model.compile(optimizer, loss, metrics)

In [ ]:
assert model.trainable

Nun trainieren wir das Modell und geben die Accuracy für die Testdaten aus.

In [ ]:
#Modell trainieren
history = model.fit(X_train, y_train, epochs=10, validation_split=0.3)

#Trainiertes Modell auswerten
test_loss, test_acc = model.evaluate (X_test, y_test)
print('Test accuracy:', test_acc)

In [ ]:
import matplotlib.pyplot as plt
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Die Ergebnisse sind etwas ernüchternd. Wir sehen, dass bereits nach einer Epoche das Modell ähnlich gute Ergebnisse erzielt, wie die logistische Regression, diese Ergebnisse aber durch weitere Optimierungschritte nicht mehr verbessert werden.
Sie können nun versuchen, die Ergebnisse durch Ändern des Modells zu verbessern, alledings sind die Erfolgsaussichten gering.
Dieser Datensatz ist zu "einfach" und eignet sich nicht unbedingt für ein MLP.

### Hausnummern erkennen

Wir wollen daher noch einen Weiteren Datensatz betrachten, der deutlich komplexer ist.
Es handelt sich um Fotos, bzw. um Bildausschnitte die einzelne Ziffern von Hausnummern zeigen.
Damit ähneln die Daten dem MNIST Datensatz.
Da es sich um (Farb-) Fotos handelt, die zudem noch recht verrauscht sind, ist das Problem, die Ziffern zu erkennen, aber deutlich schwieriger.

Wir laden zunächst die Bidler von der URL http://ufldl.stanford.edu/housenumbers herunter.
Details zum Datensatz finden Sie in [1].

In [ ]:
import pandas as pd
import os
import tarfile
import urllib.request


url = [f"http://ufldl.stanford.edu/housenumbers/{n}_32x32.mat" for n in ("train", "test")]
dfile = [f"./{n}_32x32.mat" for n in ("train", "test")]


for i in range(len(url)):
    if not os.path.isfile(dfile[i]):
        urllib.request.urlretrieve(url[i], dfile[i])

Die Daten liegen im `.mat`-Format vor, dass zumeist in Matlab verwendet wird.
Wir importieren die Daten über die Funktion `scipy.io.loadmat` und extrahieren dann die Attribute und Labels jeweils aus den Test- und Trainings-Daten.

In [ ]:
from scipy.io import loadmat
train_raw = loadmat('./train_32x32.mat')
test_raw = loadmat('./test_32x32.mat')
                   
train_images = np.array(train_raw['X'])
test_images = np.array(test_raw['X'])

train_labels = train_raw['y']
test_labels = test_raw['y']
                   
print(train_images.shape)
print(test_images.shape)

Wenn Sie sich die Dimension der Datensätze ansehen, stellen Sie fest, dass die Daten unpassend strukturiert sind.
In den ersten Dimensionen haben wir die (RGB) Pixel der einzelnen Bilder, in der letzten Dimension die einzelnen Bilder.
Daher sortieren wir die Dimensionen, bzw. die Axen unserer Datensätze um, sodass die erste Dimension dem Index eines Bildes entspricht.

In [ ]:
# Fix the axes of the images

train_images = np.moveaxis(train_images, -1, 0)
test_images = np.moveaxis(test_images, -1, 0)

print(train_images.shape)
print(test_images.shape)

Nun können wir ein zufälliges Bild ausgeben:

In [ ]:
import matplotlib.pyplot as plt
import random 
# Plot a random image and its label

plt.imshow(train_images[random.randint(0,len(train_images))])
plt.show()

print('Label: ', train_labels[13529])

In [ ]:
train_images = train_images.astype('float64')
test_images = test_images.astype('float64')

train_labels = train_labels.astype('int64')
test_labels = test_labels.astype('int64')

train_images /= 255.0
test_images /= 255.0

train_labels -= 1
test_labels -= 1

In [ ]:
assert test_labels.max()==9, """Die Label stimmen nicht.
Das kann passieren, wenn Sie eine Zelle doppelt ausgeführt haben.
Importieren Sie den Datensatz noch einmal"""

**Aufgabe: Stellen Sie ein sequenziellen Keras Model auf. Das Modell soll 2 Schichten haben.
Die erste Schicht soll 128 Neuronen besitzen und ReLU als Aktivierungsfunktion verwenden.
Die zweite Schicht soll 10 Neuronen besitzen und als Aktivierungsfunktion die Softmax-Funktion verwenden.
Bei der Eingabeschicht orientieren Sie sich am besten an den MNIST Beispielen für Keras.**

In [ ]:
#Modell definieren

def create_model():
    model = keras.Sequential()
    # YOUR CODE HERE
    raise NotImplementedError()
    return model

model = create_model()

In [ ]:
assert len(model.layers) == 3, "Das Model soll eine Input Schicht und 2 Hidden Layers besitzen"
assert model.layers[1].output_shape[1] == 128
assert model.layers[2].output_shape[1] == 10

**Aufgabe: Wählen Sie geeignete Parameter für das Modell aus**

In [ ]:
#Modellparameter
optimizer =None
loss = None
metrics = None

# YOUR CODE HERE
raise NotImplementedError()

#Modell erzeugen
model.compile(optimizer,loss,metrics)


In [ ]:
assert model.trainable

Nun können wir das Modell trainieren.

In [ ]:
#Modell trainieren
history = model.fit(train_images, train_labels, epochs=5)

In [ ]:
#Trainiertes Modell auswerten
test_loss, test_acc = model.evaluate (test_images, test_labels)
print('Test accuracy:', test_acc)

In [ ]:
import matplotlib.pyplot as plt
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Den Kurven der Accuracy und der Kostenfunktion nach zu urteilen, kann das Modell noch weiter verbesser werden.
Um bei gleichen (Hyper-)Parametern das Modell nicht immer neu trainieren zu müssen ist es sinnvoll, die Modellparameter in einer Datei zu speichern.

**Aufgabe: Schreiben Sie den Code von oben so um, dass nur dann ein Modell erzeugt wird, wenn im aktuellen Verzeichnis keine Datei `model.h5` existiert.
Wenn Sie existiert, soll das Modell aus dieser Datei geladen werden.**

**Fitten Sie das Modell über 5 Epochen und speichern Sie das Modell danach in die Datei `model.h5`.**

**Wenn Sie die Code-Zelle erneut ausführen, sollten die Vortrainierten Parameter verwendet werden**

*Hinweis: Sie können die Methoden `keras.models.load_model` und `save` zum Laden und Speichern des Modells verwenden*

In [ ]:

mfile = "./model.h5"

# YOUR CODE HERE
raise NotImplementedError()


### Referenzen

[1] Yuval Netzer, Tao Wang, Adam Coates, Alessandro Bissacco, Bo Wu, Andrew Y. Ng. *"Reading Digits in Natural Images with Unsupervised Feature Learning"*,  NIPS Workshop on Deep Learning and Unsupervised Feature Learning 2011.
